### Importing necessary libraries

In [1]:
import bs4
import pandas as pd
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

### Mention path for chrome drivers (ensure you have the appropriate driver version)

###### download from - https://sites.google.com/a/chromium.org/chromedriver/downloads

mention correct PATH

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

In [3]:
# reading the csv
game_prices = pd.read_csv('gamePrices.csv')
game_prices.head()

,Game_Name
0,NieR Replicant™ ver.1.22474487139...
1,Valheim
2,It Takes Two
3,DEATH STRANDING
4,Titanfall® 2


In [4]:
# list of games to search
listOfGames = list(game_prices['Game_Name'])
type(listOfGames)

list

In [5]:
# getting today's date
todays_date = str(datetime.now())
todays_date = todays_date[:10]
todays_date

'2021-04-25'

In [6]:
# filling values for today's column with '' (nothing)
game_prices[todays_date] = ''

In [7]:

driver = webdriver.Chrome(PATH)
driver.get("https://store.steampowered.com/search/?sort_by=_ASC&filter=topsellers")

for each in listOfGames:
    # clearing search
    search = driver.find_element_by_class_name("text").clear()
    # entering game to search
    search = driver.find_element_by_class_name("text")
    search.send_keys(each)
    # clicking enter key after typing in the necessary game name in the search bar
    search.send_keys(Keys.ENTER)
    # retriving url
    my_url = driver.current_url
    # web scraping URL using beautiful soup and extracting necessary information
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    game = str(page_soup.findAll("div", {"class":"responsive_search_name_combined"}))
    game_price = re.findall(r'A\$ \d+.\d+', game)
    game_price = game_price[-1]
    game_prices.loc[game_prices['Game_Name'] == each, todays_date] = game_price
    # going back
    driver.back()
    
# closing web browser
driver.quit()

In [8]:
# updating the csv file
game_prices.to_csv('gamePrices.csv', index = False)